In [65]:
import requests
import json
import pandas as pd
from bs4 import BeautifulSoup

Reusable function to change column names to upper

In [66]:
def upper_columns(data):
    columns = data
    myList = []
    for name in columns:
        myList.append(name.upper())
    data.columns = myList

### Census.gov API Call

In [67]:
response = None
url1 = 'https://api.census.gov/data/timeseries/poverty/saipe?get=NAME,SAEPOVRTALL_PT,SAEPOVRT0_17_PT,SAEMHI_PT,YEAR&for=state:*'
url2 ='https://api.census.gov/data/timeseries/poverty/saipe?get=SAEPOVRTALL_PT,SAEPOVRT0_17_PT,SAEMHI_PT,YEAR&for=us'

url3 = 'https://api.census.gov/data/timeseries/poverty/saipe?get=NAME,SAEPOVRTALL_PT,SAEPOVRT0_17_PT,SAEMHI_PT,YEAR&for=county:*'

poverty_response1 = requests.get(url1)
poverty_response2 = requests.get(url2)
poverty_response3 = requests.get(url3)

Convert our api response to json

In [68]:
poverty_response_json1 = poverty_response1.json()
poverty_response_json2 = poverty_response2.json()
poverty_response_json3 = poverty_response3.json()

load Poverty Data in DF

In [69]:
poverty_state = pd.DataFrame(poverty_response_json1[1:], columns=poverty_response_json1[0])
poverty_nat = pd.DataFrame(poverty_response_json2[1:], columns=poverty_response_json2[0])
poverty_county = pd.DataFrame(poverty_response_json3[1:], columns=poverty_response_json3[0])

Create DF for State_Name + state_number

In [70]:
state_name_df = poverty_state[['NAME','state']]
state_name_df.rename(columns={'NAME': "STATE_NAME"}, inplace=True)
poverty_county_state = state_name_df.merge(poverty_county, on='state')
poverty_county_state = poverty_county_state.drop_duplicates()


C:\Users\kendr\AppData\Local\Temp\ipykernel_36584\4236907917.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  state_name_df.rename(columns={'NAME': "STATE_NAME"}, inplace=True)


In [71]:
print(poverty_state.columns)
print(poverty_nat.columns)
print(poverty_county.columns)

Index(['NAME', 'SAEPOVRTALL_PT', 'SAEPOVRT0_17_PT', 'SAEMHI_PT', 'YEAR',
       'state'],
      dtype='object')
Index(['SAEPOVRTALL_PT', 'SAEPOVRT0_17_PT', 'SAEMHI_PT', 'YEAR', 'us'], dtype='object')
Index(['NAME', 'SAEPOVRTALL_PT', 'SAEPOVRT0_17_PT', 'SAEMHI_PT', 'YEAR',
       'state', 'county'],
      dtype='object')


Drop & Rename Columns for Merge

In [72]:
poverty_state = poverty_state.drop(columns='state')
poverty_nat = poverty_nat.drop(columns='us')

poverty_state = poverty_state.rename(columns={"NAME": "STATE", "SAEPOVRTALL_PT": "PR_ALL", "SAEPOVRT0_17_PT": "PR_YOUTH", 'SAEMHI_PT': 'MED_HH_INCOME' })
poverty_nat = poverty_nat.rename(columns={"NAME": "STATE", "SAEPOVRTALL_PT": "PR_ALL", "SAEPOVRT0_17_PT": "PR_YOUTH", 'SAEMHI_PT': 'MED_HH_INCOME' })


poverty_county_state = poverty_county_state.drop(columns='state')
poverty_county_state = poverty_county_state.rename(columns={"NAME": "COUNTY", "SAEPOVRTALL_PT": "PR_ALL", "SAEPOVRT0_17_PT": "PR_YOUTH", 'SAEMHI_PT': 'MED_HH_INCOME' })
poverty_county_state = poverty_county_state.drop(columns='county')

In [73]:
poverty_state = poverty_state.astype({'PR_ALL': float, 'PR_YOUTH': float, 'MED_HH_INCOME': float, 'YEAR': int})
poverty_nat = poverty_nat.astype({'PR_ALL': float, 'PR_YOUTH': float, 'MED_HH_INCOME': float, 'YEAR': int})
poverty_county_state = poverty_county_state.astype({'PR_ALL': float, 'PR_YOUTH': float, 'MED_HH_INCOME': float, 'YEAR': int})

In [74]:
print(poverty_state.dtypes)
print(poverty_nat.dtypes)
print(poverty_county_state.dtypes)

STATE             object
PR_ALL           float64
PR_YOUTH         float64
MED_HH_INCOME    float64
YEAR               int32
dtype: object
PR_ALL           float64
PR_YOUTH         float64
MED_HH_INCOME    float64
YEAR               int32
dtype: object
STATE_NAME        object
COUNTY            object
PR_ALL           float64
PR_YOUTH         float64
MED_HH_INCOME    float64
YEAR               int32
dtype: object


In [75]:
# poverty_nat = poverty_nat[poverty_nat['YEAR'] >= 2017]
# poverty_state = poverty_state[poverty_state['YEAR'] >= 2017]

In [76]:
poverty_state = poverty_state[poverty_state['STATE'] != 'Guam'] 
poverty_state = poverty_state[poverty_state['STATE'] != 'Puerto Rico'] 
poverty_state = poverty_state[poverty_state['STATE'] != 'Virgin Islands']
poverty_state = poverty_state[poverty_state['STATE'] != 'District of Columbia']

Load CSV for LAT/LONG/POPULATION

In [77]:
uscounties_df = pd.read_csv("./data/uscounties.csv")

Capitalize column names

In [78]:
upper_columns(uscounties_df)

Merge County Poverty Data with Locations/Population(2020)

In [80]:
county_locs_df = poverty_county_state.merge(uscounties_df, how='left', left_on=['STATE_NAME', 'COUNTY'], right_on=['STATE_NAME', 'COUNTY_FULL'])

In [81]:
county_locs_df.head(3)

,STATE_NAME,COUNTY_x,PR_ALL,PR_YOUTH,MED_HH_INCOME,YEAR,COUNTY_y,COUNTY_ASCII,COUNTY_FULL,COUNTY_FIPS,STATE_ID,LAT,LNG,POPULATION
0,Alabama,Autauga County,14.3,20.2,26898.0,1989,Autauga,Autauga,Autauga County,1001.0,AL,32.5349,-86.6427,58239.0
1,Alabama,Baldwin County,13.2,18.6,24043.0,1989,Baldwin,Baldwin,Baldwin County,1003.0,AL,30.7276,-87.7226,227131.0
2,Alabama,Barbour County,26.2,38.2,18673.0,1989,Barbour,Barbour,Barbour County,1005.0,AL,31.8696,-85.3932,25259.0


Remove unnecessary columns

In [82]:
county_final = county_locs_df[['STATE_NAME','YEAR','PR_ALL','PR_YOUTH','MED_HH_INCOME','COUNTY_FULL','LAT','LNG','POPULATION']]

Remove population data from all year except 2020

In [83]:
for i in range(len(county_final)):
    if county_final.loc[i, "YEAR"] != 2020:
        county_final.loc[i,'POPULATION'] = None

# National salary data

In [84]:
national2001_df = pd.read_excel("data/national_M2001_dl.xls")
national2002_df = pd.read_excel("data/national_M2002_dl.xls")
national2003_df = pd.read_excel("data/national_M2003_dl.xls")
national2004_df = pd.read_excel("data/national_M2004_dl.xls")
national2005_df = pd.read_excel("data/national_M2005_dl.xls")
national2006_df = pd.read_excel("data/national_M2006_dl.xls")
national2007_df = pd.read_excel("data/national_M2007_dl.xls")
national2008_df = pd.read_excel("data/national_M2008_dl.xls")
national2009_df = pd.read_excel("data/national_M2009_dl.xls")
national2010_df = pd.read_excel("data/national_M2010_dl.xls")
national2011_df = pd.read_excel("data/national_M2011_dl.xls")
national2012_df = pd.read_excel("data/national_M2012_dl.xls")
national2013_df = pd.read_excel("data/national_M2013_dl.xls")
national2014_df = pd.read_excel("data/national_M2014_dl.xlsx")
national2015_df = pd.read_excel("data/national_M2015_dl.xlsx")
national2016_df = pd.read_excel("data/national_M2016_dl.xlsx")
national2017_df = pd.read_excel("data/national_M2017_dl.xlsx")
national2018_df = pd.read_excel("data/national_M2018_dl.xlsx")
national2019_df = pd.read_excel("data/national_M2019_dl.xlsx")
national2020_df = pd.read_excel("data/national_M2020_dl.xlsx")
national2021_df = pd.read_excel("data/national_M2021_dl.xlsx")
national2022_df = pd.read_excel("data/national_M2022_dl.xlsx")

Add Year columns to data

In [85]:
national2002_df['Year'] = 2002
national2003_df['Year'] = 2003
national2004_df['Year'] = 2004
national2005_df['Year'] = 2005
national2006_df['Year'] = 2006
national2007_df['Year'] = 2007
national2008_df['Year'] = 2008
national2009_df['Year'] = 2009
national2010_df['Year'] = 2010
national2011_df['Year'] = 2011
national2012_df['Year'] = 2012
national2013_df['Year'] = 2013
national2014_df['Year'] = 2014
national2015_df['Year'] = 2015
national2016_df['Year'] = 2016
national2017_df['Year'] = 2017
national2018_df['Year'] = 2018
national2019_df['Year'] = 2019
national2020_df['Year'] = 2020
national2021_df['Year'] = 2021
national2022_df['Year'] = 2022

Capitalize column names

In [86]:
upper_columns(national2001_df)
upper_columns(national2002_df)
upper_columns(national2003_df)
upper_columns(national2004_df)
upper_columns(national2005_df)
upper_columns(national2006_df)
upper_columns(national2007_df)
upper_columns(national2008_df)
upper_columns(national2009_df)
upper_columns(national2010_df)
upper_columns(national2011_df)
upper_columns(national2012_df)
upper_columns(national2013_df)
upper_columns(national2014_df)
upper_columns(national2015_df)
upper_columns(national2016_df)
upper_columns(national2017_df)
upper_columns(national2018_df)
upper_columns(national2019_df)
upper_columns(national2020_df)
upper_columns(national2021_df)
upper_columns(national2022_df)

Rename columns that dont match

In [87]:
n2001_df = national2001_df.rename(columns={ 'GROUP': 'OCC_GROUP', 'H_WPCT10' : 'H_PCT10', 'H_WPCT25' : 'H_PCT25', 'H_WPCT75' : 'H_PCT75', 'H_WPCT90' : 'H_PCT90', 'A_WPCT10' : 'A_PCT10', 'A_WPCT25' : 'A_PCT25', 'A_WPCT75' : 'A_PCT75', 'A_WPCT90' : 'A_PCT90'})
n2002_df = national2002_df.rename(columns={ 'GROUP': 'OCC_GROUP', 'H_WPCT10' : 'H_PCT10', 'H_WPCT25' : 'H_PCT25', 'H_WPCT75' : 'H_PCT75', 'H_WPCT90' : 'H_PCT90', 'A_WPCT10' : 'A_PCT10', 'A_WPCT25' : 'A_PCT25', 'A_WPCT75' : 'A_PCT75', 'A_WPCT90' : 'A_PCT90'})
n2003_df = national2003_df.rename(columns={ 'GROUP': 'OCC_GROUP'})
n2004_df = national2004_df.rename(columns={ 'GROUP': 'OCC_GROUP'})
n2005_df = national2005_df.rename(columns={ 'GROUP': 'OCC_GROUP'})
n2006_df = national2006_df.rename(columns={ 'GROUP': 'OCC_GROUP'})
n2007_df = national2007_df.rename(columns={ 'GROUP': 'OCC_GROUP'})
n2008_df = national2008_df.rename(columns={ 'GROUP': 'OCC_GROUP'})
n2009_df = national2009_df.rename(columns={'GROUP': 'OCC_GROUP'})
n2010_df = national2010_df.rename(columns={'LOC QUOTIENT': 'LOC_QUOTIENT', 'GROUP': 'OCC_GROUP'})
n2011_df = national2011_df.rename(columns={'LOC_Q': 'LOC_QUOTIENT', 'GROUP': 'OCC_GROUP'})
n2012_df = national2012_df.rename(columns={'LOC_Q': 'LOC_QUOTIENT'})
n2013_df = national2013_df.rename(columns={'LOC_Q': 'LOC_QUOTIENT'})
n2014_df = national2014_df.rename(columns={'LOC_Q': 'LOC_QUOTIENT'})
n2015_df = national2015_df.rename(columns={'LOC_Q': 'LOC_QUOTIENT'})
n2016_df = national2016_df.rename(columns={'LOC_Q': 'LOC_QUOTIENT'})
n2017_df = national2017_df.rename(columns={'LOC_Q': 'LOC_QUOTIENT'})
n2018_df = national2018_df.rename(columns={'LOC_Q': 'LOC_QUOTIENT'})
n2019_df = national2019_df.rename(columns={'O_GROUP': 'OCC_GROUP', 'AREA_TITLE': 'STATE'})
n2020_df = national2020_df.rename(columns={'O_GROUP': 'OCC_GROUP', 'AREA_TITLE': 'STATE'})
n2021_df = national2021_df.rename(columns={'O_GROUP': 'OCC_GROUP', 'AREA_TITLE': 'STATE'})
n2022_df = national2022_df.rename(columns={'O_GROUP': 'OCC_GROUP', 'AREA_TITLE': 'STATE'})

Filter columns and set correct column order

In [88]:
n2001_ordered = n2001_df[[
    'OCC_CODE', 'OCC_TITLE', 'OCC_GROUP', 'TOT_EMP',
   'EMP_PRSE', 'H_MEAN', 'A_MEAN', 'MEAN_PRSE',
   'H_PCT10', 'H_PCT25', 'H_MEDIAN', 'H_PCT75', 'H_PCT90', 'A_PCT10',
   'A_PCT25', 'A_MEDIAN', 'A_PCT75', 'A_PCT90', 'YEAR']]
n2002_ordered = n2002_df[[
    'OCC_CODE', 'OCC_GROUP', 'TOT_EMP',
   'EMP_PRSE', 'H_MEAN', 'A_MEAN', 'MEAN_PRSE',
   'H_PCT10', 'H_PCT25', 'H_MEDIAN', 'H_PCT75', 'H_PCT90', 'A_PCT10',
   'A_PCT25', 'A_MEDIAN', 'A_PCT75', 'A_PCT90', 'YEAR']]
n2003_ordered = n2003_df[[
  'OCC_CODE', 'OCC_TITLE', 'OCC_GROUP', 'TOT_EMP',
   'EMP_PRSE', 'H_MEAN', 'A_MEAN', 'MEAN_PRSE',
   'H_PCT10', 'H_PCT25', 'H_MEDIAN', 'H_PCT75', 'H_PCT90', 'A_PCT10',
   'A_PCT25', 'A_MEDIAN', 'A_PCT75', 'A_PCT90', 'YEAR']]
n2004_ordered = n2004_df[[
    'OCC_CODE', 'OCC_TITLE', 'OCC_GROUP', 'TOT_EMP',
   'EMP_PRSE', 'H_MEAN', 'A_MEAN', 'MEAN_PRSE',
   'H_PCT10', 'H_PCT25', 'H_MEDIAN', 'H_PCT75', 'H_PCT90', 'A_PCT10',
   'A_PCT25', 'A_MEDIAN', 'A_PCT75', 'A_PCT90', 'YEAR']]
n2005_ordered = n2005_df[[
    'OCC_CODE', 'OCC_TITLE', 'OCC_GROUP', 'TOT_EMP',
   'EMP_PRSE', 'H_MEAN', 'A_MEAN', 'MEAN_PRSE',
   'H_PCT10', 'H_PCT25', 'H_MEDIAN', 'H_PCT75', 'H_PCT90', 'A_PCT10',
   'A_PCT25', 'A_MEDIAN', 'A_PCT75', 'A_PCT90', 'YEAR']]
n2006_ordered = n2006_df[[
    'OCC_CODE', 'OCC_TITLE', 'OCC_GROUP', 'TOT_EMP',
   'EMP_PRSE', 'H_MEAN', 'A_MEAN', 'MEAN_PRSE',
   'H_PCT10', 'H_PCT25', 'H_MEDIAN', 'H_PCT75', 'H_PCT90', 'A_PCT10',
   'A_PCT25', 'A_MEDIAN', 'A_PCT75', 'A_PCT90', 'YEAR']]
n2007_ordered = n2007_df[[
    'OCC_CODE', 'OCC_TITLE', 'OCC_GROUP', 'TOT_EMP',
   'EMP_PRSE', 'H_MEAN', 'A_MEAN', 'MEAN_PRSE',
   'H_PCT10', 'H_PCT25', 'H_MEDIAN', 'H_PCT75', 'H_PCT90', 'A_PCT10',
   'A_PCT25', 'A_MEDIAN', 'A_PCT75', 'A_PCT90', 'YEAR']]
n2008_ordered = n2008_df[[
    'OCC_CODE', 'OCC_TITLE', 'OCC_GROUP', 'TOT_EMP',
   'EMP_PRSE', 'H_MEAN', 'A_MEAN', 'MEAN_PRSE',
   'H_PCT10', 'H_PCT25', 'H_MEDIAN', 'H_PCT75', 'H_PCT90', 'A_PCT10',
   'A_PCT25', 'A_MEDIAN', 'A_PCT75', 'A_PCT90', 'YEAR']]
n2009_ordered = n2009_df[[
    'OCC_CODE', 'OCC_TITLE', 'OCC_GROUP', 'TOT_EMP',
   'EMP_PRSE', 'H_MEAN', 'A_MEAN', 'MEAN_PRSE',
   'H_PCT10', 'H_PCT25', 'H_MEDIAN', 'H_PCT75', 'H_PCT90', 'A_PCT10',
   'A_PCT25', 'A_MEDIAN', 'A_PCT75', 'A_PCT90', 'YEAR']]
n2010_ordered = n2010_df[[
    'OCC_CODE', 'OCC_TITLE', 'OCC_GROUP', 'TOT_EMP',
   'EMP_PRSE', 'H_MEAN', 'A_MEAN', 'MEAN_PRSE',
   'H_PCT10', 'H_PCT25', 'H_MEDIAN', 'H_PCT75', 'H_PCT90', 'A_PCT10',
   'A_PCT25', 'A_MEDIAN', 'A_PCT75', 'A_PCT90', 'YEAR']]
n2011_ordered = n2011_df[[
    'OCC_CODE', 'OCC_TITLE', 'OCC_GROUP', 'TOT_EMP',
   'EMP_PRSE',  'H_MEAN', 'A_MEAN', 'MEAN_PRSE',
   'H_PCT10', 'H_PCT25', 'H_MEDIAN', 'H_PCT75', 'H_PCT90', 'A_PCT10',
   'A_PCT25', 'A_MEDIAN', 'A_PCT75', 'A_PCT90', 'YEAR']]
n2012_ordered = n2012_df[[
    'OCC_CODE', 'OCC_TITLE', 'OCC_GROUP', 'TOT_EMP',
   'EMP_PRSE',  'H_MEAN', 'A_MEAN', 'MEAN_PRSE',
   'H_PCT10', 'H_PCT25', 'H_MEDIAN', 'H_PCT75', 'H_PCT90', 'A_PCT10',
   'A_PCT25', 'A_MEDIAN', 'A_PCT75', 'A_PCT90', 'YEAR']]
n2013_ordered = n2013_df[[
    'OCC_CODE', 'OCC_TITLE', 'OCC_GROUP', 'TOT_EMP',
   'EMP_PRSE',  'H_MEAN', 'A_MEAN', 'MEAN_PRSE',
   'H_PCT10', 'H_PCT25', 'H_MEDIAN', 'H_PCT75', 'H_PCT90', 'A_PCT10',
   'A_PCT25', 'A_MEDIAN', 'A_PCT75', 'A_PCT90', 'YEAR']]
n2014_ordered = n2014_df[[
   'OCC_CODE', 'OCC_TITLE', 'OCC_GROUP', 'TOT_EMP',
   'EMP_PRSE',  'H_MEAN', 'A_MEAN', 'MEAN_PRSE',
   'H_PCT10', 'H_PCT25', 'H_MEDIAN', 'H_PCT75', 'H_PCT90', 'A_PCT10',
   'A_PCT25', 'A_MEDIAN', 'A_PCT75', 'A_PCT90', 'YEAR']]
n2015_ordered = n2015_df[[
    'OCC_CODE', 'OCC_TITLE', 'OCC_GROUP', 'TOT_EMP',
   'EMP_PRSE',  'H_MEAN', 'A_MEAN', 'MEAN_PRSE',
   'H_PCT10', 'H_PCT25', 'H_MEDIAN', 'H_PCT75', 'H_PCT90', 'A_PCT10',
   'A_PCT25', 'A_MEDIAN', 'A_PCT75', 'A_PCT90', 'YEAR']]
n2016_ordered = n2016_df[[
    'OCC_CODE', 'OCC_TITLE', 'OCC_GROUP', 'TOT_EMP',
   'EMP_PRSE',  'H_MEAN', 'A_MEAN', 'MEAN_PRSE',
   'H_PCT10', 'H_PCT25', 'H_MEDIAN', 'H_PCT75', 'H_PCT90', 'A_PCT10',
   'A_PCT25', 'A_MEDIAN', 'A_PCT75', 'A_PCT90', 'YEAR']]
n2017_ordered = n2017_df[[
    'OCC_CODE', 'OCC_TITLE', 'OCC_GROUP', 'TOT_EMP',
   'EMP_PRSE',  'H_MEAN', 'A_MEAN', 'MEAN_PRSE',
   'H_PCT10', 'H_PCT25', 'H_MEDIAN', 'H_PCT75', 'H_PCT90', 'A_PCT10',
   'A_PCT25', 'A_MEDIAN', 'A_PCT75', 'A_PCT90', 'YEAR']]
n2018_ordered = n2018_df[[
    'OCC_CODE', 'OCC_TITLE', 'OCC_GROUP', 'TOT_EMP',
   'EMP_PRSE',  'H_MEAN', 'A_MEAN', 'MEAN_PRSE',
  'H_PCT10', 'H_PCT25', 'H_MEDIAN', 'H_PCT75', 'H_PCT90', 'A_PCT10',
   'A_PCT25', 'A_MEDIAN', 'A_PCT75', 'A_PCT90', 'YEAR']]
n2019_ordered = n2019_df[[
    'OCC_CODE', 'OCC_TITLE', 'OCC_GROUP', 'TOT_EMP',
   'EMP_PRSE',  'H_MEAN', 'A_MEAN', 'MEAN_PRSE',
   'H_PCT10', 'H_PCT25', 'H_MEDIAN', 'H_PCT75', 'H_PCT90', 'A_PCT10',
   'A_PCT25', 'A_MEDIAN', 'A_PCT75', 'A_PCT90', 'YEAR']]
n2020_ordered = n2020_df[[
   'OCC_CODE', 'OCC_TITLE', 'OCC_GROUP', 'TOT_EMP',
   'EMP_PRSE',  'H_MEAN', 'A_MEAN', 'MEAN_PRSE',
   'H_PCT10', 'H_PCT25', 'H_MEDIAN', 'H_PCT75', 'H_PCT90', 'A_PCT10',
   'A_PCT25', 'A_MEDIAN', 'A_PCT75', 'A_PCT90', 'YEAR']]
n2021_ordered = n2021_df[[
    'OCC_CODE', 'OCC_TITLE', 'OCC_GROUP', 'TOT_EMP',
   'EMP_PRSE',  'H_MEAN', 'A_MEAN', 'MEAN_PRSE',
   'H_PCT10', 'H_PCT25', 'H_MEDIAN', 'H_PCT75', 'H_PCT90', 'A_PCT10',
   'A_PCT25', 'A_MEDIAN', 'A_PCT75', 'A_PCT90', 'YEAR']]
n2022_ordered = n2022_df[[
     'OCC_CODE', 'OCC_TITLE', 'OCC_GROUP', 'TOT_EMP',
    'EMP_PRSE',  'H_MEAN', 'A_MEAN', 'MEAN_PRSE',
    'H_PCT10', 'H_PCT25', 'H_MEDIAN', 'H_PCT75', 'H_PCT90', 'A_PCT10',
    'A_PCT25', 'A_MEDIAN', 'A_PCT75', 'A_PCT90', 'YEAR']]

Concat all National data into one df

In [89]:
national_df = pd.concat([n2001_ordered,n2002_ordered,n2003_ordered,n2004_ordered,
                         n2005_ordered,n2006_ordered, n2007_ordered,n2008_ordered,
                         n2009_ordered,n2010_ordered, n2011_ordered,n2012_ordered,
                         n2013_ordered,n2014_ordered,n2015_ordered,n2016_ordered,
                         n2017_ordered,n2018_ordered,n2019_ordered,n2020_ordered,
                         n2021_ordered,n2022_ordered ], axis=0)
national_df.shape

(23969, 19)

In [90]:
national_df = national_df[[
    'OCC_TITLE', 'OCC_GROUP', 'TOT_EMP', 'H_MEAN',
    'A_MEAN', 'H_MEDIAN', 'A_MEDIAN', 'YEAR']]
national_df

,OCC_TITLE,OCC_GROUP,TOT_EMP,H_MEAN,A_MEAN,H_MEDIAN,A_MEDIAN,YEAR
0,All Occupations,NaN,127980410,16.35,34020,13.01,27060,2001
1,Management Occupations,major,7212360,34.04,70800,30.88,64220,2001
2,Chief Executives,NaN,455930,51.77,107670,57.91,120450,2001
3,General and Operations Managers,NaN,2064220,35.37,73570,31.25,65000,2001
4,Legislators,NaN,67400,13.54,28170,7.05,14650,2001
...,...,...,...,...,...,...,...,...
1397,Refuse and Recyclable Material Collectors,detailed,132240,21.91,45560,20.94,43540,2022
1398,"Tank Car, Truck, and Ship Loaders",broad,12470,28.21,58680,25.93,53930,2022
1399,"Tank Car, Truck, and Ship Loaders",detailed,12470,28.21,58680,25.93,53930,2022
1400,Miscellaneous Material Moving Workers,broad,24880,20.81,43290,18.65,38800,2022


Remove missing data:

In [91]:
cols = list(national_df.columns)
for col in cols: 
    national_df = national_df[national_df[col] != '#']
    national_df = national_df[national_df[col] != '*']
    national_df = national_df[national_df[col] != '**']
national_df.head(1)

,OCC_TITLE,OCC_GROUP,TOT_EMP,H_MEAN,A_MEAN,H_MEDIAN,A_MEDIAN,YEAR
0,All Occupations,NaN,127980410,16.35,34020,13.01,27060,2001


Convert types:

In [92]:
print(national_df.columns)
print(national_df.dtypes)

Index(['OCC_TITLE', 'OCC_GROUP', 'TOT_EMP', 'H_MEAN', 'A_MEAN', 'H_MEDIAN',
       'A_MEDIAN', 'YEAR'],
      dtype='object')
OCC_TITLE    object
OCC_GROUP    object
TOT_EMP      object
H_MEAN       object
A_MEAN       object
H_MEDIAN     object
A_MEDIAN     object
YEAR          int64
dtype: object


In [93]:
nat_final_df = national_df.astype({'TOT_EMP': int, 'H_MEAN': float, 'A_MEAN': float, 'H_MEDIAN': float, 'A_MEDIAN': float, 'YEAR': int})
print(national_df.dtypes)

OCC_TITLE    object
OCC_GROUP    object
TOT_EMP      object
H_MEAN       object
A_MEAN       object
H_MEDIAN     object
A_MEDIAN     object
YEAR          int64
dtype: object


# State salary data

Read in state salary data from excel

In [94]:
state2001_df = pd.read_excel("data/state_M2001_dl.xls")
state2002_df = pd.read_excel("data/state_M2002_dl.xls")
state2003_df = pd.read_excel("data/state_M2003_dl.xls")
state2004_df = pd.read_excel("data/state_M2004_dl.xls")
state2005_df = pd.read_excel("data/state_M2005_dl.xls")
state2006_df = pd.read_excel("data/state_M2006_dl.xls")
state2007_df = pd.read_excel("data/state_M2007_dl.xls")
state2008_df = pd.read_excel("data/state_M2008_dl.xls")
state2009_df = pd.read_excel("data/state_M2009_dl.xls")
state2010_df = pd.read_excel("data/state_M2010_dl.xls")
state2011_df = pd.read_excel("data/state_M2011_dl.xls")
state2012_df = pd.read_excel("data/state_M2012_dl.xls")
state2013_df = pd.read_excel("data/state_M2013_dl.xls")
state2014_df = pd.read_excel("data/state_M2014_dl.xlsx")
state2015_df = pd.read_excel("data/state_M2015_dl.xlsx")
state2016_df = pd.read_excel("data/state_M2016_dl.xlsx")
state2017_df = pd.read_excel("data/state_M2017_dl.xlsx")
state2018_df = pd.read_excel("data/state_M2018_dl.xlsx")
state2019_df = pd.read_excel("data/state_M2019_dl.xlsx")
state2020_df = pd.read_excel("data/state_M2020_dl.xlsx")
state2021_df = pd.read_excel("data/state_M2021_dl.xlsx")
state2022_df = pd.read_excel("data/state_M2022_dl.xlsx")

Add Year Column

In [95]:
state2002_df['Year'] = 2002
state2003_df['Year'] = 2003
state2004_df['Year'] = 2004
state2005_df['Year'] = 2005
state2006_df['Year'] = 2006
state2007_df['Year'] = 2007
state2008_df['Year'] = 2008
state2009_df['Year'] = 2009
state2010_df['Year'] = 2010
state2011_df['Year'] = 2011
state2012_df['Year'] = 2012
state2013_df['Year'] = 2013
state2014_df['Year'] = 2014
state2015_df['Year'] = 2015
state2016_df['Year'] = 2016
state2017_df['Year'] = 2017
state2018_df['Year'] = 2018
state2019_df['Year'] = 2019
state2020_df['Year'] = 2020
state2021_df['Year'] = 2021
state2022_df['Year'] = 2022

Capitalize column names

In [96]:
upper_columns(state2001_df)
upper_columns(state2002_df)
upper_columns(state2003_df)
upper_columns(state2004_df)
upper_columns(state2005_df)
upper_columns(state2006_df)
upper_columns(state2007_df)
upper_columns(state2008_df)
upper_columns(state2009_df)
upper_columns(state2010_df)
upper_columns(state2011_df)
upper_columns(state2012_df)
upper_columns(state2013_df)
upper_columns(state2014_df)
upper_columns(state2015_df)
upper_columns(state2016_df)
upper_columns(state2017_df)
upper_columns(state2018_df)
upper_columns(state2019_df)
upper_columns(state2020_df)
upper_columns(state2021_df)
upper_columns(state2022_df)

Rename columns that are unmatched

In [97]:
s2001_df = state2001_df.rename(columns={ 'GROUP': 'OCC_GROUP', 'H_WPCT10' : 'H_PCT10', 'H_WPCT25' : 'H_PCT25', 'H_WPCT75' : 'H_PCT75', 'H_WPCT90' : 'H_PCT90', 'A_WPCT10' : 'A_PCT10', 'A_WPCT25' : 'A_PCT25', 'A_WPCT75' : 'A_PCT75', 'A_WPCT90' : 'A_PCT90'})
s2002_df = state2002_df.rename(columns={ 'GROUP': 'OCC_GROUP', 'H_WPCT10' : 'H_PCT10', 'H_WPCT25' : 'H_PCT25', 'H_WPCT75' : 'H_PCT75', 'H_WPCT90' : 'H_PCT90', 'A_WPCT10' : 'A_PCT10', 'A_WPCT25' : 'A_PCT25', 'A_WPCT75' : 'A_PCT75', 'A_WPCT90' : 'A_PCT90'})
s2003_df = state2003_df.rename(columns={ 'GROUP': 'OCC_GROUP'})
s2004_df = state2004_df.rename(columns={ 'GROUP': 'OCC_GROUP'})
s2005_df = state2005_df.rename(columns={ 'GROUP': 'OCC_GROUP'})
s2006_df = state2006_df.rename(columns={ 'GROUP': 'OCC_GROUP'})
s2007_df = state2007_df.rename(columns={ 'GROUP': 'OCC_GROUP'})
s2008_df = state2008_df.rename(columns={ 'GROUP': 'OCC_GROUP'})
s2009_df = state2009_df.rename(columns={'GROUP': 'OCC_GROUP'})
s2010_df = state2010_df.rename(columns={'LOC QUOTIENT': 'LOC_QUOTIENT', 'GROUP': 'OCC_GROUP'})
s2011_df = state2011_df.rename(columns={'LOC_Q': 'LOC_QUOTIENT', 'GROUP': 'OCC_GROUP'})
s2012_df = state2012_df.rename(columns={'LOC_Q': 'LOC_QUOTIENT'})
s2013_df = state2013_df.rename(columns={'LOC_Q': 'LOC_QUOTIENT'})
s2014_df = state2014_df.rename(columns={'LOC_Q': 'LOC_QUOTIENT'})
s2015_df = state2015_df.rename(columns={'LOC_Q': 'LOC_QUOTIENT'})
s2016_df = state2016_df.rename(columns={'LOC_Q': 'LOC_QUOTIENT'})
s2017_df = state2017_df.rename(columns={'LOC_Q': 'LOC_QUOTIENT'})
s2018_df = state2018_df.rename(columns={'LOC_Q': 'LOC_QUOTIENT'})
s2019_df = state2019_df.rename(columns={'O_GROUP': 'OCC_GROUP', 'AREA_TITLE': 'STATE'})
s2020_df = state2020_df.rename(columns={'O_GROUP': 'OCC_GROUP', 'AREA_TITLE': 'STATE'})
s2021_df = state2021_df.rename(columns={'O_GROUP': 'OCC_GROUP', 'AREA_TITLE': 'STATE'})
s2022_df = state2022_df.rename(columns={'O_GROUP': 'OCC_GROUP', 'AREA_TITLE': 'STATE'})


Filter columns and set correct column order

In [98]:
s2001_ordered = s2001_df[[
    'AREA', 'STATE', 'OCC_CODE', 'OCC_TITLE', 'OCC_GROUP', 'TOT_EMP',
    'EMP_PRSE', 'H_MEAN', 'A_MEAN', 'MEAN_PRSE',
    'H_PCT10', 'H_PCT25', 'H_MEDIAN', 'H_PCT75', 'H_PCT90', 'A_PCT10',
    'A_PCT25', 'A_MEDIAN', 'A_PCT75', 'A_PCT90', 'YEAR']]
s2002_ordered = s2002_df[[
    'AREA', 'STATE', 'OCC_CODE', 'OCC_TITLE', 'OCC_GROUP', 'TOT_EMP',
    'EMP_PRSE',  'H_MEAN', 'A_MEAN', 'MEAN_PRSE',
    'H_PCT10', 'H_PCT25', 'H_MEDIAN', 'H_PCT75', 'H_PCT90', 'A_PCT10',
    'A_PCT25', 'A_MEDIAN', 'A_PCT75', 'A_PCT90', 'YEAR']]
s2003_ordered = s2003_df[[
    'AREA', 'STATE', 'OCC_CODE', 'OCC_TITLE', 'OCC_GROUP', 'TOT_EMP',
    'EMP_PRSE',  'H_MEAN', 'A_MEAN', 'MEAN_PRSE',
    'H_PCT10', 'H_PCT25', 'H_MEDIAN', 'H_PCT75', 'H_PCT90', 'A_PCT10',
    'A_PCT25', 'A_MEDIAN', 'A_PCT75', 'A_PCT90', 'YEAR']]
s2004_ordered = s2004_df[[
    'AREA', 'STATE', 'OCC_CODE', 'OCC_TITLE', 'OCC_GROUP', 'TOT_EMP',
    'EMP_PRSE',  'H_MEAN', 'A_MEAN', 'MEAN_PRSE',
    'H_PCT10', 'H_PCT25', 'H_MEDIAN', 'H_PCT75', 'H_PCT90', 'A_PCT10',
    'A_PCT25', 'A_MEDIAN', 'A_PCT75', 'A_PCT90', 'YEAR']]
s2005_ordered = s2005_df[[
    'AREA', 'STATE', 'OCC_CODE', 'OCC_TITLE', 'OCC_GROUP', 'TOT_EMP',
    'EMP_PRSE',  'H_MEAN', 'A_MEAN', 'MEAN_PRSE',
    'H_PCT10', 'H_PCT25', 'H_MEDIAN', 'H_PCT75', 'H_PCT90', 'A_PCT10',
    'A_PCT25', 'A_MEDIAN', 'A_PCT75', 'A_PCT90', 'YEAR']]
s2006_ordered = s2006_df[[
    'AREA', 'STATE', 'OCC_CODE', 'OCC_TITLE', 'OCC_GROUP', 'TOT_EMP',
    'EMP_PRSE',  'H_MEAN', 'A_MEAN', 'MEAN_PRSE',
    'H_PCT10', 'H_PCT25', 'H_MEDIAN', 'H_PCT75', 'H_PCT90', 'A_PCT10',
    'A_PCT25', 'A_MEDIAN', 'A_PCT75', 'A_PCT90', 'YEAR']]
s2007_ordered = s2007_df[[
    'AREA', 'STATE', 'OCC_CODE', 'OCC_TITLE', 'OCC_GROUP', 'TOT_EMP',
    'EMP_PRSE',  'H_MEAN', 'A_MEAN', 'MEAN_PRSE',
    'H_PCT10', 'H_PCT25', 'H_MEDIAN', 'H_PCT75', 'H_PCT90', 'A_PCT10',
    'A_PCT25', 'A_MEDIAN', 'A_PCT75', 'A_PCT90', 'YEAR']]
s2008_ordered = s2008_df[[
    'AREA', 'STATE', 'OCC_CODE', 'OCC_TITLE', 'OCC_GROUP', 'TOT_EMP',
    'EMP_PRSE', 'H_MEAN', 'A_MEAN', 'MEAN_PRSE',
    'H_PCT10', 'H_PCT25', 'H_MEDIAN', 'H_PCT75', 'H_PCT90', 'A_PCT10',
    'A_PCT25', 'A_MEDIAN', 'A_PCT75', 'A_PCT90', 'YEAR']]
s2009_ordered = s2009_df[[
    'AREA', 'STATE', 'OCC_CODE', 'OCC_TITLE', 'OCC_GROUP', 'TOT_EMP',
    'EMP_PRSE', 'H_MEAN', 'A_MEAN', 'MEAN_PRSE',
    'H_PCT10', 'H_PCT25', 'H_MEDIAN', 'H_PCT75', 'H_PCT90', 'A_PCT10',
    'A_PCT25', 'A_MEDIAN', 'A_PCT75', 'A_PCT90', 'YEAR']]
s2010_ordered = s2010_df[[
    'AREA', 'STATE', 'OCC_CODE', 'OCC_TITLE', 'OCC_GROUP', 'TOT_EMP',
    'EMP_PRSE', 'JOBS_1000', 'LOC_QUOTIENT', 'H_MEAN', 'A_MEAN', 'MEAN_PRSE',
    'H_PCT10', 'H_PCT25', 'H_MEDIAN', 'H_PCT75', 'H_PCT90', 'A_PCT10',
    'A_PCT25', 'A_MEDIAN', 'A_PCT75', 'A_PCT90', 'YEAR']]
s2011_ordered = s2011_df[[
    'AREA', 'STATE', 'OCC_CODE', 'OCC_TITLE', 'OCC_GROUP', 'TOT_EMP',
    'EMP_PRSE', 'JOBS_1000', 'LOC_QUOTIENT', 'H_MEAN', 'A_MEAN', 'MEAN_PRSE',
    'H_PCT10', 'H_PCT25', 'H_MEDIAN', 'H_PCT75', 'H_PCT90', 'A_PCT10',
    'A_PCT25', 'A_MEDIAN', 'A_PCT75', 'A_PCT90', 'YEAR']]
s2012_ordered = s2012_df[[
    'AREA', 'STATE', 'OCC_CODE', 'OCC_TITLE', 'OCC_GROUP', 'TOT_EMP',
    'EMP_PRSE', 'JOBS_1000', 'LOC_QUOTIENT', 'H_MEAN', 'A_MEAN', 'MEAN_PRSE',
    'H_PCT10', 'H_PCT25', 'H_MEDIAN', 'H_PCT75', 'H_PCT90', 'A_PCT10',
    'A_PCT25', 'A_MEDIAN', 'A_PCT75', 'A_PCT90', 'YEAR']]
s2013_ordered = s2013_df[[
    'AREA', 'STATE', 'OCC_CODE', 'OCC_TITLE', 'OCC_GROUP', 'TOT_EMP',
    'EMP_PRSE', 'JOBS_1000', 'LOC_QUOTIENT', 'H_MEAN', 'A_MEAN', 'MEAN_PRSE',
    'H_PCT10', 'H_PCT25', 'H_MEDIAN', 'H_PCT75', 'H_PCT90', 'A_PCT10',
    'A_PCT25', 'A_MEDIAN', 'A_PCT75', 'A_PCT90', 'YEAR']]
s2014_ordered = s2014_df[[
    'AREA', 'STATE', 'OCC_CODE', 'OCC_TITLE', 'OCC_GROUP', 'TOT_EMP',
    'EMP_PRSE', 'JOBS_1000', 'LOC_QUOTIENT', 'H_MEAN', 'A_MEAN', 'MEAN_PRSE',
    'H_PCT10', 'H_PCT25', 'H_MEDIAN', 'H_PCT75', 'H_PCT90', 'A_PCT10',
    'A_PCT25', 'A_MEDIAN', 'A_PCT75', 'A_PCT90', 'YEAR']]
s2015_ordered = s2015_df[[
    'AREA', 'STATE', 'OCC_CODE', 'OCC_TITLE', 'OCC_GROUP', 'TOT_EMP',
    'EMP_PRSE', 'JOBS_1000', 'LOC_QUOTIENT', 'H_MEAN', 'A_MEAN', 'MEAN_PRSE',
    'H_PCT10', 'H_PCT25', 'H_MEDIAN', 'H_PCT75', 'H_PCT90', 'A_PCT10',
    'A_PCT25', 'A_MEDIAN', 'A_PCT75', 'A_PCT90', 'YEAR']]
s2016_ordered = s2016_df[[
    'AREA', 'STATE', 'OCC_CODE', 'OCC_TITLE', 'OCC_GROUP', 'TOT_EMP',
    'EMP_PRSE', 'JOBS_1000', 'LOC_QUOTIENT', 'H_MEAN', 'A_MEAN', 'MEAN_PRSE',
    'H_PCT10', 'H_PCT25', 'H_MEDIAN', 'H_PCT75', 'H_PCT90', 'A_PCT10',
    'A_PCT25', 'A_MEDIAN', 'A_PCT75', 'A_PCT90', 'YEAR']]
s2017_ordered = s2017_df[[
    'AREA', 'STATE', 'OCC_CODE', 'OCC_TITLE', 'OCC_GROUP', 'TOT_EMP',
    'EMP_PRSE', 'JOBS_1000', 'LOC_QUOTIENT', 'H_MEAN', 'A_MEAN', 'MEAN_PRSE',
    'H_PCT10', 'H_PCT25', 'H_MEDIAN', 'H_PCT75', 'H_PCT90', 'A_PCT10',
    'A_PCT25', 'A_MEDIAN', 'A_PCT75', 'A_PCT90', 'YEAR']]
s2018_ordered = s2018_df[[
    'AREA', 'STATE', 'OCC_CODE', 'OCC_TITLE', 'OCC_GROUP', 'TOT_EMP',
    'EMP_PRSE', 'JOBS_1000', 'LOC_QUOTIENT', 'H_MEAN', 'A_MEAN', 'MEAN_PRSE',
    'H_PCT10', 'H_PCT25', 'H_MEDIAN', 'H_PCT75', 'H_PCT90', 'A_PCT10',
    'A_PCT25', 'A_MEDIAN', 'A_PCT75', 'A_PCT90', 'YEAR']]
s2019_ordered = s2019_df[[
    'AREA', 'STATE', 'OCC_CODE', 'OCC_TITLE', 'OCC_GROUP', 'TOT_EMP',
    'EMP_PRSE', 'JOBS_1000', 'LOC_QUOTIENT', 'H_MEAN', 'A_MEAN', 'MEAN_PRSE',
    'H_PCT10', 'H_PCT25', 'H_MEDIAN', 'H_PCT75', 'H_PCT90', 'A_PCT10',
    'A_PCT25', 'A_MEDIAN', 'A_PCT75', 'A_PCT90', 'YEAR']]
s2020_ordered = s2020_df[[
    'AREA', 'STATE', 'OCC_CODE', 'OCC_TITLE', 'OCC_GROUP', 'TOT_EMP',
    'EMP_PRSE', 'JOBS_1000', 'LOC_QUOTIENT', 'H_MEAN', 'A_MEAN', 'MEAN_PRSE',
    'H_PCT10', 'H_PCT25', 'H_MEDIAN', 'H_PCT75', 'H_PCT90', 'A_PCT10',
    'A_PCT25', 'A_MEDIAN', 'A_PCT75', 'A_PCT90', 'YEAR']]
s2021_ordered = s2021_df[[
    'AREA', 'STATE', 'OCC_CODE', 'OCC_TITLE', 'OCC_GROUP', 'TOT_EMP',
    'EMP_PRSE', 'JOBS_1000', 'LOC_QUOTIENT', 'H_MEAN', 'A_MEAN', 'MEAN_PRSE',
    'H_PCT10', 'H_PCT25', 'H_MEDIAN', 'H_PCT75', 'H_PCT90', 'A_PCT10',
    'A_PCT25', 'A_MEDIAN', 'A_PCT75', 'A_PCT90', 'YEAR']]
s2022_ordered = s2022_df[[
    'AREA', 'STATE', 'OCC_CODE', 'OCC_TITLE', 'OCC_GROUP', 'TOT_EMP',
    'EMP_PRSE', 'JOBS_1000', 'LOC_QUOTIENT', 'H_MEAN', 'A_MEAN', 'MEAN_PRSE',
    'H_PCT10', 'H_PCT25', 'H_MEDIAN', 'H_PCT75', 'H_PCT90', 'A_PCT10',
    'A_PCT25', 'A_MEDIAN', 'A_PCT75', 'A_PCT90', 'YEAR']]

Concat all State data into one df

In [99]:
state_df = pd.concat([s2001_ordered, s2002_ordered, s2003_ordered,s2004_ordered,s2005_ordered,s2006_ordered
                      ,s2007_ordered,s2008_ordered,s2009_ordered,s2010_ordered,s2011_ordered,s2012_ordered,
                      s2013_ordered,s2014_ordered,s2015_ordered,s2016_ordered,s2017_ordered,s2018_ordered,
                      s2019_ordered,s2020_ordered,s2021_ordered,s2022_ordered], axis=0)
state_df.shape

(798959, 23)

Remove non-state territories

In [100]:
state_df = state_df[state_df['STATE'] != 'Guam'] 
state_df = state_df[state_df['STATE'] != 'Puerto Rico'] 
state_df = state_df[state_df['STATE'] != 'Virgin Islands']
state_df = state_df[state_df['STATE'] != 'District of Columbia']
state_df.columns

Index(['AREA', 'STATE', 'OCC_CODE', 'OCC_TITLE', 'OCC_GROUP', 'TOT_EMP',
       'EMP_PRSE', 'H_MEAN', 'A_MEAN', 'MEAN_PRSE', 'H_PCT10', 'H_PCT25',
       'H_MEDIAN', 'H_PCT75', 'H_PCT90', 'A_PCT10', 'A_PCT25', 'A_MEDIAN',
       'A_PCT75', 'A_PCT90', 'YEAR', 'JOBS_1000', 'LOC_QUOTIENT'],
      dtype='object')

Filter out unnecessary columns

In [101]:
state_df = state_df[[
    'AREA', 'STATE', 'OCC_TITLE', 'OCC_GROUP', 'TOT_EMP',
    'JOBS_1000', 'LOC_QUOTIENT', 'H_MEAN', 'A_MEAN',
    'H_MEDIAN', 'A_MEDIAN', 'YEAR']]

Remove missing data

In [102]:
cols = list(state_df.columns)
for col in cols:
    state_df = state_df[state_df[col] != '#']
    state_df = state_df[state_df[col] != '*']
    state_df = state_df[state_df[col] != '**']

Convert types

In [103]:
state_final_df = state_df.astype({
    'TOT_EMP': int, 'JOBS_1000': float, 'LOC_QUOTIENT': float, 
    'H_MEAN': float, 'A_MEAN': float, 'H_MEDIAN': float, 
    'A_MEDIAN': float, 'YEAR': int})

In [104]:
print(poverty_nat.isna().sum())
print('2')
print(poverty_state.isna().sum())
print('3')
print(nat_final_df.isna().sum())
print('4')
print(state_final_df.isna().sum())

PR_ALL           0
PR_YOUTH         0
MED_HH_INCOME    0
YEAR             0
dtype: int64
2
STATE            0
PR_ALL           0
PR_YOUTH         0
MED_HH_INCOME    0
YEAR             0
dtype: int64
3
OCC_TITLE     673
OCC_GROUP    7790
TOT_EMP         0
H_MEAN          0
A_MEAN          0
H_MEDIAN        0
A_MEDIAN        0
YEAR            0
dtype: int64
4
AREA                 0
STATE                0
OCC_TITLE            0
OCC_GROUP       315171
TOT_EMP              0
JOBS_1000       265637
LOC_QUOTIENT    265637
H_MEAN               0
A_MEAN               0
H_MEDIAN             0
A_MEDIAN             0
YEAR                 0
dtype: int64


In [105]:
print(poverty_nat.dtypes)
print(poverty_state.dtypes)
print(nat_final_df.dtypes)
print(state_final_df.dtypes)
print(poverty_nat.shape)
print(poverty_state.shape)
print(nat_final_df.shape)
print(state_final_df.shape)

PR_ALL           float64
PR_YOUTH         float64
MED_HH_INCOME    float64
YEAR               int32
dtype: object
STATE             object
PR_ALL           float64
PR_YOUTH         float64
MED_HH_INCOME    float64
YEAR               int32
dtype: object
OCC_TITLE     object
OCC_GROUP     object
TOT_EMP        int32
H_MEAN       float64
A_MEAN       float64
H_MEDIAN     float64
A_MEDIAN     float64
YEAR           int32
dtype: object
AREA              int64
STATE            object
OCC_TITLE        object
OCC_GROUP        object
TOT_EMP           int32
JOBS_1000       float64
LOC_QUOTIENT    float64
H_MEAN          float64
A_MEAN          float64
H_MEDIAN        float64
A_MEDIAN        float64
YEAR              int32
dtype: object
(29, 4)
(1450, 5)
(22155, 8)
(670962, 12)


In [106]:
display(poverty_state.columns)
display(poverty_state.dtypes)

display(state_final_df.columns)
display(state_final_df.dtypes)

display(county_final.columns)
display(county_final.dtypes)


Index(['STATE', 'PR_ALL', 'PR_YOUTH', 'MED_HH_INCOME', 'YEAR'], dtype='object')

STATE             object
PR_ALL           float64
PR_YOUTH         float64
MED_HH_INCOME    float64
YEAR               int32
dtype: object

Index(['AREA', 'STATE', 'OCC_TITLE', 'OCC_GROUP', 'TOT_EMP', 'JOBS_1000',
       'LOC_QUOTIENT', 'H_MEAN', 'A_MEAN', 'H_MEDIAN', 'A_MEDIAN', 'YEAR'],
      dtype='object')

AREA              int64
STATE            object
OCC_TITLE        object
OCC_GROUP        object
TOT_EMP           int32
JOBS_1000       float64
LOC_QUOTIENT    float64
H_MEAN          float64
A_MEAN          float64
H_MEDIAN        float64
A_MEDIAN        float64
YEAR              int32
dtype: object

In [110]:
print(state_df.shape)
print(poverty_state.shape)
print(county_final.shape)

(670962, 12)
(1450, 5)
(91113, 9)


Merge the Povery & Salary Data at for the State Level

In [108]:
states_merged = state_df.merge(poverty_state, on=['STATE', 'YEAR'])
states_merged.shape

(639115, 15)

Write ouput files as CSV

In [109]:
poverty_state.to_csv('./output/state_poverty.csv', index=False)
poverty_nat.to_csv('./output/national_poverty.csv', index=False)
nat_final_df.to_csv('./output/national_salary.csv', index=False)
state_final_df.to_csv('./output/state_salary.csv', index=False)
states_merged.to_csv('./output/state_poverty_and_income.csv', index=False)
county_final.to_csv('./output/county_poverty.csv', index=False)